In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR
import holodeck.anisotropy as anis

import numpy as np
import healpy as hp
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import cmath

Read in sam data

In [ ]:
nn=346
sspath = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/'
hdfname = sspath+'sam_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][nn]
hc_bg = ssfile['hc_bg'][nn]
fobs = ssfile['fobs'][:]
# dfobs = ssfile['dfobs'][:]
ssfile.close()

shape=hc_ss.shape
nfreqs = shape[-3]
nreals = shape[-2]
nloudest = shape[-1]
print('nfreqs=%d, nreals=%d, nloudest=%d' % (nfreqs, nreals, nloudest))

## Spherical Harmonic Functions
$$ a_\ell^m = \frac{4\pi}{N} \sum_{i=1}^N \bar{Y_\ell}^m (\lambda_i, \theta_i)f(\lambda_i, \theta_i) $$
$0 \leq \ell \leq \ell_\mathrm{max}$, $-\ell \leq m \leq \ell $
For real harmonics only, $m=0$

In [ ]:
def calc_alm(ell, emm, func, nside):
    """ 
    
    func : 
        function of ipix or theta,phi
    """
    npix = hp.nside2npix(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    Ylm = sp.special.sph_harm(emm, ell, theta, phi)
    integrand = np.sum(np.conjugate(Ylm) * func)
    alm = 4*np.pi/npix * integrand
    return alm


$$C_\ell = \frac{1}{2\ell+1} \sum_m |a_\ell^m|^2$$

In [ ]:

def calc_Cl(ell, func, nside):
    sum = 0
    for emm in range(-ell, ell+1):
        # print('l=%d, m=%d' % (ell, emm))
        alm = calc_alm(ell, emm, func, nside)
        sum += alm*np.conjugate(alm)
    Cl = 1/(2*ell + 1) * sum
    # if Cl.imag != 0:
    #     # print(f"warning! {Cl=} contains imaginary numbers")
    #     # print(f"{alm=}")
    #     return Cl
    return Cl.real

In [ ]:
nside = 2
npix = hp.nside2npix(nside)
theta, phi = hp.pix2ang(nside, np.arange(npix))
moll = anis.healpix_map(hc_ss[0:1,0:1], hc_bg[0:1,0:1], nside)
moll = np.squeeze(moll)
print(theta.shape)
print(phi.shape)
# print(theta,phi)
print(moll.shape)

In [ ]:
lmax = 6
Cl_arr = np.zeros(lmax+1)
for ll in range(lmax+1):
    Cl_arr[ll] = calc_Cl(ll, moll, nside).real
    print('C_%d = %.2e' % (ll, Cl_arr[ll]))

# ff=1, rr=0, ll=1

In [ ]:
rr = 0
ll=1

xx = np.arange(lmax+1)
yy = Cl_arr

# colors = cm.rainbow(np.linspace(0,1,yy))

fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
ax.plot(xx, yy)
ax.set_title('Analytic' )
fig.tight_layout()

In [ ]:
ff = 1
rr = 0


nsides = np.array([4, 8, 16, 32, 64])
colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

for ii, nside in enumerate(nsides):
    npix = hp.nside2npix(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    moll = anis.healpix_map(hc_ss[ff:ff+1,rr:rr+1], hc_bg[ff:ff+1,rr:rr+1], nside)
    moll = np.squeeze(moll)

    lmax = 8
    Cl_arr = np.zeros(lmax+1)
    for ll in range(lmax+1):
        Cl_arr[ll] = calc_Cl(ll, moll, nside).real
        # print('C_%d = %.2e' % (ll, Cl_arr[ll]))

    xx = np.arange(lmax+1)
    yy = Cl_arr
    ax.plot(xx, yy, label = 'Drake Eqs. (1) & (3), nside=%d' % nside,
            color=colors[ii])

ax.legend()
ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
fig.tight_layout()


In [ ]:
ff=1
rr=1

nsides = np.array([8, 16, 32, 64])
colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

for ii, nside in enumerate(nsides):
    print('\n nside=%d' % nside)
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    moll = anis.healpix_map_oldhc2(hc_ss[ff:ff+1,rr:rr+1], hc_bg[ff:ff+1,rr:rr+1], nside)
    moll = moll/area

    lmax = 8
    Cl_arr = np.zeros(lmax+1)
    for ll in range(lmax+1):
        Cl_arr[ll] = calc_Cl(ll, moll, nside).real
        # print('C_%d = %.2e' % (ll, Cl_arr[ll]))

    xx = np.arange(lmax+1)
    yy = Cl_arr
    ax.plot(xx, yy, label = 'From Eq. (3), nside=%d' % nside,
            color=colors[ii])
ax.legend()
ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
fig.tight_layout()


# Compare to healpy results

In [ ]:
save_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/healpix/varying_nside_testnpz'
hcfile = np.load(save_loc+'/hc2dOm_A.npz')
print(hcfile.files)
Cl2dOm_arrs = hcfile['Cl2dOm_arrs']
moll_arrs = [hcfile['moll2dOm_ss08'], hcfile['moll2dOm_tt16'], 
                      hcfile['moll2dOm_tt32'], hcfile['moll2dOm_tt64']]
# nsides=hcfile['nsides']

hcfile.close()
print(Cl2dOm_arrs.shape)
    

In [ ]:
print(moll_arrs[0].shape)

In [ ]:
ff = 1
rr = 0
lmax=6
xx = np.arange(lmax+1)

nsides = np.array([8, 16, 32, 64])
colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

for ii, nside in enumerate(nsides):
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    # moll = anis.healpix_map(hc_ss[ff:ff+1,rr:rr+1], hc_bg[ff:ff+1,rr:rr+1], nside)
    # moll = np.squeeze(moll)
    moll = moll_arrs[ii][ff,rr]

    Cl_arr = np.zeros(lmax+1)
    Cl_hpy = hp.anafast(moll, lmax=lmax)
    for ll in range(lmax+1):
        Cl_arr[ll] = calc_Cl(ll, moll, nside)
        # print('C_%d = %.2e' % (ll, Cl_arr[ll]))


    yy = Cl_arr
    ax.plot(xx, yy, label = 'From Eq. (3), nside=%d' % nside,
            color=colors[ii], linestyle='-', marker='o')
    # ax.plot(xx, Cl2dOm_arrs[ii,ff,rr,:], color=colors[ii], linestyle=':',
    #         label=('hp.anafast, nside=%d' % (nside)))
    ax.plot(xx, Cl_hpy[:], color=colors[ii], linestyle=':', marker='x',
            label=('hp.anafast, nside=%d' % (nside)))

ax.legend()
ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
fig.tight_layout()


# ClC0 Comparison

In [ ]:
ff = 1
rr = 0
lmax=6
xx = np.arange(1,lmax+1)

nsides = np.array([8, 16, 32, 64])
colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

for ii, nside in enumerate(nsides):
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    moll = moll_arrs[ii][ff,rr]

    Cl_arr = np.zeros(lmax+1)
    Cl_hpy = hp.anafast(moll, lmax=lmax)
    for ll in range(lmax+1):
        Cl_arr[ll] = calc_Cl(ll, moll, nside)
        # print('C_%d = %.2e' % (ll, Cl_arr[ll]))


    y1 = Cl_arr[1:]/Cl_arr[0]
    y2 = Cl_hpy[1:]/Cl_hpy[0]
    ax.plot(xx, y1, label = 'From Eq. (3), nside=%d' % nside,
            color=colors[ii])
    # ax.plot(xx, Cl2dOm_arrs[ii,ff,rr,:], color=colors[ii], linestyle=':',
    #         label=('hp.anafast, nside=%d' % (nside)))
    ax.plot(xx, y2, color=colors[ii], linestyle=':',
            label=('hp.anafast, nside=%d' % (nside)))

ax.legend()
ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
fig.tight_layout()


alm values

In [ ]:
ff = 1
rr = 0
mm = 0
lmax=6

nsides = np.array([8, 16, 32, 64])
colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

for ii, nside in enumerate(nsides):
    npix = hp.nside2npix(nside)
    area = hp.nside2pixarea(nside)
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    moll = anis.healpix_map(hc_ss[ff:ff+1,rr:rr+1], hc_bg[ff:ff+1,rr:rr+1], nside)
    moll = np.squeeze(moll)**2/area

    lmax = 6
    xx = np.arange(lmax+1)

    for ll in range(lmax+1):
        emm_arr = np.arange(-lmax, lmax+1)
        print(f"{emm_arr}")
        for mm, emm in enumerate(emm_arr):
            alm_arr = np.zeros(len(emm_arr))
            alm_arr[mm] = calc_alm(ll, emm, moll, nside)
            ax.scatter(xx[ll], alm_arr[mm])
        # print('C_%d = %.2e' % (ll, Cl_arr[ll]))

    # yy = alm_arr
    # ax.scatter(xx, yy, label = '$a_\ell^m$, nside=%d, $m$=%d' % (nside,mm),
    #         color=colors[ii])
for ii, nside in enumerate(nsides):
    ax.plot(xx, Cl2dOm_arrs[ii,ff,rr,:], color=colors[ii], linestyle=':',
            label=('hp.anafast, nside=%d' % (nside)))

ax.legend()
ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
fig.tight_layout()


In [ ]:
ax.legend()
fig

# Different realizations and frequencies

plot_Cl_vs_l

In [ ]:
def plot_Cl_vs_l(ff, rr, lmax):
    xx = np.arange(lmax+1)

    nsides = np.array([8, 16, 32, 64])
    colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
    fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear',
                         figsize=(6,4))

    for ii, nside in enumerate(nsides):
        npix = hp.nside2npix(nside)
        area = hp.nside2pixarea(nside)
        theta, phi = hp.pix2ang(nside, np.arange(npix))
        # moll = anis.healpix_map(hc_ss[ff:ff+1,rr:rr+1], hc_bg[ff:ff+1,rr:rr+1], nside)
        # moll = np.squeeze(moll)
        moll = moll_arrs[ii][ff,rr]

        Cl_arr = np.zeros(lmax+1)
        Cl_hpy = hp.anafast(moll, lmax=lmax)
        for ll in range(lmax+1):
            Cl_arr[ll] = calc_Cl(ll, moll, nside)
            # print('C_%d = %.2e' % (ll, Cl_arr[ll]))


        yy = Cl_arr
        ax.plot(xx, yy, label = 'From Eq. (3), nside=%d' % nside,
                color=colors[ii], linestyle='-', marker='o')
        # ax.plot(xx, Cl2dOm_arrs[ii,ff,rr,:], color=colors[ii], linestyle=':',
        #         label=('hp.anafast, nside=%d' % (nside)))
        ax.plot(xx, Cl_hpy[:], color=colors[ii], linestyle=':', marker='x',
                label=('hp.anafast, nside=%d' % (nside)))

    ax.legend(fontsize=10, loc='upper right')
    ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
    fig.tight_layout()
    return fig

plot_ClC0_vs_l

In [ ]:
def plot_ClC0_vs_l(ff, rr, lmax):
    xx = np.arange(1,lmax+1)

    nsides = np.array([8, 16, 32, 64])
    colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
    fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear',
                         figsize=(6,4))

    for ii, nside in enumerate(nsides):
        npix = hp.nside2npix(nside)
        area = hp.nside2pixarea(nside)
        theta, phi = hp.pix2ang(nside, np.arange(npix))
        moll = moll_arrs[ii][ff,rr]

        Cl_arr = np.zeros(lmax+1)
        Cl_hpy = hp.anafast(moll, lmax=lmax)
        for ll in range(lmax+1):
            Cl_arr[ll] = calc_Cl(ll, moll, nside)
            # print('C_%d = %.2e' % (ll, Cl_arr[ll]))

 
        y1 = Cl_arr[1:]/Cl_arr[0]
        y2 = Cl_hpy[1:]/Cl_hpy[0]
        ax.plot(xx, y1, label = 'From Eq. (3), nside=%d' % nside,
                color=colors[ii])
        # ax.plot(xx, Cl2dOm_arrs[ii,ff,rr,:], color=colors[ii], linestyle=':',
        #         label=('hp.anafast, nside=%d' % (nside)))
        ax.plot(xx, y2, color=colors[ii], linestyle=':',
                label=('hp.anafast, nside=%d' % (nside)))

    ax.legend(fontsize=8, loc='upper right')
    ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
    fig.tight_layout()
    return fig


In [ ]:
def plot_C0_vs_nside(ff, rr, lmax):

    nsides = np.array([8, 16, 32, 64])
    xx = np.arange(len(nsides))
    # colors = cm.rainbow(np.linspace(0, 1, len(nsides)))
    fig, ax = plot.figax(xlabel='$N_\mathrm{side}$', ylabel='$C_{\ell=0}$', xscale='log',
                         figsize=(5,3))

    # Calculate Cl from Eqs.
    moll = moll_arrs[ii][ff,rr]
    C0_arr = np.zeros(len(nsides))
    C0_hpy = np.zeros(len(nsides))
    for nn, nside in enumerate(nsides):
        moll = moll_arrs[nn][ff,rr]
        C0_arr[nn] = calc_Cl(0, moll, nside)
        C0_hpy[nn] = hp.anafast(moll, lmax=lmax)[0]

    y1 = C0_arr
    y2 = C0_hpy
    ax.plot(xx, y1, label = 'From Eq. (3)' % nside, linestyle='-')
    ax.plot(xx, y2, label=('hp.anafast' % (nside)), linestyle=':', )

    ax.legend(fontsize=12, loc='lower right')
    ax.set_title('Sample %d, Realization %d, $f$=%.2f/yr, $h_c^2/d\Omega$' % (nn, rr, fobs[ff]*YR, ))
    fig.tight_layout()
    return fig


C0 vs nside

In [ ]:
for rr in range(10):
    fig = plot_C0_vs_nside(1, rr, lmax)


Cl Comparison

In [ ]:
for rr in range(10):
    fig = plot_Cl_vs_l(1, rr, lmax)


ClC0 comparison

In [ ]:
for rr in range(10):
    fig = plot_ClC0_vs_l(1, rr, lmax)
